In [24]:
from bs4 import BeautifulSoup
import requests 
import time
import pandas as pd

In [8]:
URL = "https://www.imsdb.com/all%20scripts/"
r = requests.get(URL) 
soup = BeautifulSoup(r.content, 'html5lib')
movies = soup.find_all("p")
movie_pages = {m.find("a").text: m.find("a").get("href") for m in movies}

In [9]:
URL = "https://www.imsdb.com/Movie%20Scripts/Ace%20Ventura:%20Pet%20Detective%20Script.html"
print(URL)
r = requests.get(URL) 
soup = BeautifulSoup(r.content, 'html5lib')

https://www.imsdb.com/Movie%20Scripts/Ace%20Ventura:%20Pet%20Detective%20Script.html


In [21]:
script_pages = dict()
for name in movie_pages:
    time.sleep(0.1)
    URL = "https://www.imsdb.com" + movie_pages[name].replace(" ","%20")
    r = requests.get(URL) 
    soup = BeautifulSoup(r.content, 'html5lib')
    
    link = soup.find_all("p")
    if len(link) > 0:
        link = link[-1]
        
        if link.text.strip().endswith("Script"):
            script_pages[name] = link.find("a").get("href")
        else:
            print(name, link)
            print()
    else:
        print(name)
        print()

A.I. <p><b>Giljonnys Dias da Silva</b> <a href="mailto:giljonnysdias@outlook.com" title="E-mail User"><img border="0" src="/images/emailuser.gif"/></a>   <img src="/images/rating/10-stars.gif"/> (10 out of 10 )<br/>Really a good movie which portraits technological reality. I have already watched this movie many times.</p>

Batman and Robin <p><b>bruce wayne</b> <a href="mailto:blackfire1118@live.com" title="E-mail User"><img border="0" src="/images/emailuser.gif"/></a>   <img src="/images/rating/1-stars.gif"/> (1 out of 10 )<br/>This movie is an absolute disaster. I am amazed how anybody in this movie still has a career. George Clooney brought shame to Michael Keaton, Christian Bale and even Val Kilmer. Arnold Schwarzenegger was the worst casting choice of the whole movie. If there was a zero choice on here I definitely would have picked it. This was overall the biggest crime in cinema history.</p>

Batman Begins <p><b>Seabass</b> <img src="/images/rating/9-stars.gif"/> (9 out of 10 )<

In [22]:
len(script_pages)

1180

In [23]:
script_pages = {name:script_pages[name] for name in script_pages if script_pages[name].startswith("/scripts/") and script_pages[name].endswith(".html")}

In [29]:
pd.DataFrame(list(script_pages.items()), columns=["name","url"]).set_index("name").to_csv("script_urls.csv")

In [106]:
names = list(script_pages.keys())
scripts = dict()
last_name = names[0]

In [115]:
for name in names[names.index(last_name):]:
    time.sleep(0.1)
    URL = "https://www.imsdb.com" + script_pages[name]
    r = requests.get(URL)
    soup = BeautifulSoup(r.content, 'html5lib')
    
    pre = soup.find("pre")
    td = soup.find("td", {"class":"scrtext"})
    if pre != None:
        if pre.script != None:
            pre.script.decompose()
        text = pre.text
    elif td != None:
        text = td.text
    else:
        print(name, URL)
        continue
    scripts[name] = text
    last_name = name

In [116]:
df = pd.DataFrame(list(scripts.items()), columns=["name","script"]).set_index("name")
df.to_csv("scripts.csv")

,script
name,
10 Things I Hate About You,\n\n\nTen Things I Hate About You - by Karen M...
12,\n\n \n 12 - Script\n\n\n\n\nCUT FROM BLACK\...
12 and Holding,\n \n \n ...
12 Monkeys,\n\n\nTwelve Monkeys\n\n\n\t\t\t\tTWELVE MONKE...
12 Years a Slave,\n\n\n 12 YEARS A ...


In [118]:
len(scripts)

1160

In [124]:
nospace = {name: " ".join(scripts[name].split()) for name in scripts}

In [126]:
df_nospace = pd.DataFrame(list(nospace.items()), columns=["name","script"]).set_index("name")
df_nospace.to_csv("scripts_nospace.csv")